In [2]:
import numpy, math

from turbo import TurboM
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt

In [4]:
lb = np.array(np.ones(30)*-10)
ub = np.array(np.ones(30)*10)
print(len(lb), len(ub))
print(lb)

30 30
[-10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10.]


In [5]:
def F10(x):
    dim = len(x)
    o = (
        -20 * numpy.exp(-0.2 * numpy.sqrt(numpy.sum(x ** 2) / dim))
        - numpy.exp(numpy.sum(numpy.cos(2 * math.pi * x)) / dim)
        + 20
        + numpy.exp(1)
    )
    return o

In [10]:
class Ackley:
    def __init__(self, dim=30):
        self.dim = dim
        self.lb = lb
        self.ub = ub
        
    def __call__(self, x):
        assert len(x[0]) == self.dim
        assert x[0].ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)

        #print(x)

        res = np.array([[F10(i)] for i in x])
        
        return res

In [11]:
f = Ackley(len(lb))

In [12]:
turbo_m = TurboM(
    f=f,  # Handle to objective function
    lb=f.lb,  # Numpy array specifying lower bounds
    ub=f.ub,  # Numpy array specifying upper bounds
    n_init=20,  # Number of initial bounds from an Symmetric Latin hypercube design
    max_evals=500,  # Maximum number of evaluations
    n_trust_regions=5,  # Number of trust regions
    batch_size=2,  # How large batch size TuRBO uses
    verbose=True,  # Print information from each batch
    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
    max_cholesky_size=1000,  # When we switch from Cholesky to Lanczos
    n_training_steps=50,  # Number of steps of ADAM to learn the hypers
    min_cuda=1024,  # Run on the CPU for small datasets
    device="cuda",  # "cpu" or "cuda"
    dtype="float64",  # float64 or float32
)

Using dtype = torch.float64 
Using device = cuda


In [ ]:
turbo_m.optimize()

TR-0 starting from: 14.27
TR-1 starting from: 14.06
TR-2 starting from: 14.26
TR-3 starting from: 14.47
TR-4 starting from: 13.79


cat: /work/projects/p0020541/Yixuan/spinw/software/para_results.txt: No such file or directory


100) New best @ TR-4: 13.31
104) New best @ TR-4: 13.18
108) New best @ TR-4: 12.86
110) New best @ TR-4: 11.83
142) New best @ TR-4: 11.52
146) New best @ TR-4: 10.58
150) New best @ TR-4: 10.54
158) New best @ TR-4: 10.34
162) New best @ TR-4: 9.804
164) New best @ TR-4: 9.582
168) New best @ TR-4: 9.242
184) New best @ TR-4: 9.229
186) New best @ TR-4: 8.841
196) New best @ TR-4: 8.127
200) New best @ TR-4: 8.051
202) New best @ TR-4: 7.863
208) New best @ TR-4: 7.835
216) New best @ TR-4: 7.457
228) New best @ TR-4: 7.328
232) New best @ TR-4: 7.236
234) New best @ TR-4: 7.154
236) New best @ TR-4: 6.561


In [ ]:
X = turbo_m.X  # Evaluated points
fX = turbo_m.fX  # Observed values
ind_best = np.argmin(fX)
f_best, x_best = fX[ind_best], X[ind_best, :]

print("Best value found:\n\tf(x) = %.3f\nObserved at:\n\tx = %s" % (f_best, np.around(x_best, 3)))

In [ ]:
fig = plt.figure(figsize=(7, 5))
matplotlib.rcParams.update({'font.size': 16})
plt.plot(fX, 'b.', ms=10)  # Plot all evaluated points as blue dots
plt.plot(np.minimum.accumulate(fX), 'r', lw=3)  # Plot cumulative minimum as a red line
plt.xlim([0, len(fX)])
plt.ylim([0, 30])
plt.title("17D motor function")

plt.tight_layout()
plt.savefig('motor.png', dpi=300)
plt.show()